In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
#import ipynb.fs.full.features_textos as f_textos

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train = features.llenar_nulls(df_train)
df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)

# df_train, df_test = features_de_csvs(df_train, df_test)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.2)

In [3]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_test_idf = pd.read_csv('./data/test_idf.csv')

df_train_f = pd.merge(df_train_f, df_train_idf, on= 'id', how= 'left')
df_test_f = pd.merge(df_test_f, df_test_idf, on= 'id', how= 'left')

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [4]:
list(df_train_f.columns)

['id',
 'titulo',
 'descripcion',
 'tipodepropiedad',
 'direccion',
 'ciudad',
 'provincia',
 'antiguedad',
 'habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'idzona',
 'lat',
 'lng',
 'fecha',
 'gimnasio',
 'usosmultiples',
 'piscina',
 'escuelascercanas',
 'centroscomercialescercanos',
 'precio',
 'metrostotales_log',
 'metroscubiertos_log',
 'porcentaje_metros',
 'diferencia_metros',
 'intervalo_metros_totales',
 'intervalo_metros_cubiertos',
 'metroscubiertos_bins_unif',
 'metroscubiertos_bins_perc',
 'escomercial',
 'promedio_metros_tipo_propiedad',
 'promedio_metros_cub_tipo_propiedad',
 'tipo_propiedad_compartida',
 'prop_frecuente',
 'zona',
 'top_provincia',
 'es_ciudad_centrica',
 'promedio_metros_totales_provincia',
 'promedio_metros_cubiertos_provincia',
 'anio',
 'mes',
 'dia',
 'trimestre',
 'dias_desde_datos',
 'meses_desde_datos',
 'escualas_centros_cercanos',
 'delincuencia',
 'turismo',
 'es_antigua',
 'antiguedad_bins_unif',
 'antiguedad_b

In [5]:
from sklearn.model_selection import train_test_split

class LightGBMWrapper(lgb.LGBMRegressor):
    
    def fit(self, x, y):        
        x_train, x_test, y_train, y_test = train_test_split(x, y)
        return super(LightGBMWrapper, self).fit(x_train, y_train)
    
    def predict(self, X):
        return super(LightGBMWrapper, self).predict(X, 
               num_iteration=self.best_iteration_)

hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': int(hps['num_leaves']),
    'learning_rate': hps['learning_rate'],
    'feature_fraction': hps['feature_fraction'],
    'bagging_fraction': hps['bagging_fraction'],
    'bagging_freq': int(hps['bagging_freq']),
    'max_depth': int(hps['max_depth']),
    'verbose': 0
}

lgb_m = LightGBMWrapper(**params)

## SelectKBest

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression

def selectKBest(x_train, y_train, features, k):
    selector_kbest = SelectKBest(f_regression, k)
    selector_kbest.fit_transform(x_train, y_train)
    kbest = list(x_train.columns[selector_kbest.get_support()])
    print('Cantidad de mejores features (k): ', str(len(kbest)))
    return kbest

In [7]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'lat', 'lng', 
            'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 
            'precio', 'metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros', 
            'escomercial', 'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad', 
            'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia', 'es_ciudad_centrica', 
            'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia', 'anio', 'mes', 'dia', 
            'trimestre', 'dias_desde_datos', 'meses_desde_datos', 'delincuencia', 'turismo', 'es_antigua', 
            'cantidad_inquilinos', 'tam_ambientes', 'metros_totales_normalizados', 
            'metros_cubiertos_normalizados', 'promedio_precio_provincia', 'promedio_precio_ciudad', 
            'promedio_precio_ciudad_gen', 'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona', 
            'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona', 'count_id_zona', 
            'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad', 
            'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 
            'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones', 
            'promedio_precio_habitaciones_banos_garages', 'promedio_precio_banos_garages', 
            'promedio_precio_hbg_tipo_propiedad', 'promedio_precio_hbg_tipo_propiedad_provincia', 
            'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'tipodepropiedad_Apartamento', 
            'tipodepropiedad_Bodega comercial', 'tipodepropiedad_Casa', 'tipodepropiedad_Casa en condominio', 
            'tipodepropiedad_Casa uso de suelo', 'tipodepropiedad_Departamento Compartido', 
            'tipodepropiedad_Duplex', 'tipodepropiedad_Edificio', 
            'tipodepropiedad_Inmuebles productivos urbanos', 'tipodepropiedad_Local Comercial', 
            'tipodepropiedad_Local en centro comercial', 'tipodepropiedad_Oficina comercial', 
            'tipodepropiedad_Otros', 'tipodepropiedad_Quinta Vacacional', 'tipodepropiedad_Rancho', 
            'tipodepropiedad_Terreno', 'tipodepropiedad_Terreno comercial', 'tipodepropiedad_Villa', 
            'tipodepropiedad_otro', 'provincia_Aguascalientes', 'provincia_Baja California Norte', 
            'provincia_Baja California Sur', 'provincia_Campeche', 'provincia_Chiapas', 'provincia_Chihuahua', 
            'provincia_Coahuila', 'provincia_Colima', 'provincia_Distrito Federal', 'provincia_Durango', 
            'provincia_Edo. de Mxico', 'provincia_Guanajuato', 'provincia_Guerrero', 'provincia_Hidalgo', 
            'provincia_Jalisco', 'provincia_Michoacn', 'provincia_Morelos', 'provincia_Nayarit', 
            'provincia_Nuevo Len', 'provincia_Oaxaca', 'provincia_Puebla', 'provincia_Quertaro', 
            'provincia_Quintana Roo', 'provincia_San luis Potos', 'provincia_Sinaloa', 'provincia_Sonora', 
            'provincia_Tabasco', 'provincia_Tamaulipas', 'provincia_Tlaxcala', 'provincia_Veracruz', 
            'provincia_Yucatn', 'provincia_otro', 'zona_Centro', 'zona_Norte ', 'zona_Sur'] 

print('Cantidad de features totales: ', str(len(features)))
x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

selectKBest(x_train, y_train, features, 30)

Cantidad de features totales:  120
Cantidad de mejores features (k):  30


['habitaciones',
 'garages',
 'banos',
 'metroscubiertos',
 'metrostotales',
 'metrostotales_log',
 'metroscubiertos_log',
 'top_provincia',
 'cantidad_inquilinos',
 'tam_ambientes',
 'metros_totales_normalizados',
 'metros_cubiertos_normalizados',
 'promedio_precio_provincia',
 'promedio_precio_ciudad',
 'promedio_precio_ciudad_gen',
 'varianza_precio_ciudad',
 'count_ciudad',
 'promedio_id_zona',
 'promedio_id_zona_log',
 'promedio_id_zona_gen',
 'varianza_id_zona',
 'promedio_precio_tipo_propiedad_ciudad',
 'promedio_precio_tipo_propiedad_ciudad_gen',
 'promedio_precio_habitaciones',
 'promedio_precio_habitaciones_banos_garages',
 'promedio_precio_banos_garages',
 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia',
 'puntaje',
 'provincia_Distrito Federal']

In [ ]:
# Me mata la RAM, SWAP y me deja triste
# x_train_g = SelectKBest(mutual_info_regression, k=20).fit_transform(x_train_f, y_train)

## SelectFromModel

In [ ]:
from sklearn.feature_selection import SelectFromModel

lgb_m = LightGBMWrapper(**params)
lgb_m.fit(x_train, y_train)

model = SelectFromModel(lgb_m, prefit=True)
x_train.columns[model.get_support()]

In [ ]:
from sklearn.feature_selection import SelectFromModel

features_sub = ['antiguedad', 'metroscubiertos', 'metrostotales', 
       'porcentaje_metros', 'diferencia_metros', 'mes', 'dia', 'dias_desde_datos',
       'meses_desde_datos', 'tam_ambientes', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'promedio_precio_ciudad',
       'varianza_precio_ciudad', 'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad_ciudad_gen',
       'promedio_por_mes',  'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'puntaje']

lgb_m = LightGBMWrapper(**params)
lgb_m.fit(x_train[features_sub], y_train)

model = SelectFromModel(lgb_m, prefit=True)
x_train[features_sub].columns[model.get_support()]

## Recursive Feature Elimination (RFE)

In [ ]:
df_train_f.isnull().any()

In [ ]:
from sklearn.feature_selection import RFE

features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales','lat', 'lng', 'gimnasio',
 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio', 'metrostotales_log',
 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros',
 'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc', 'escomercial', 'promedio_metros_tipo_propiedad',
 'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia',
 'es_ciudad_centrica', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia',
 'anio', 'mes', 'dia', 'trimestre','dias_desde_datos', 'meses_desde_datos','delincuencia',
 'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
 'metros_totales_normalizados', 'metros_cubiertos_normalizados', 'promedio_precio_provincia', 'promedio_provincia_log',
 'promedio_precio_ciudad', 'promedio_ciudad_log', 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad',
 'count_ciudad', 'promedio_id_zona', 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona',
 'count_id_zona', 'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen',
 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones',
 'promedio_precio_habitaciones_banos_garages', 'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia',
 'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'tipodepropiedad_Apartamento',
 'tipodepropiedad_Bodega comercial', 'tipodepropiedad_Casa', 'tipodepropiedad_Casa en condominio', 'tipodepropiedad_Casa uso de suelo',
 'tipodepropiedad_Departamento Compartido', 'tipodepropiedad_Duplex', 'tipodepropiedad_Edificio',
 'tipodepropiedad_Inmuebles productivos urbanos', 'tipodepropiedad_Local Comercial', 'tipodepropiedad_Local en centro comercial',
 'tipodepropiedad_Oficina comercial', 'tipodepropiedad_Otros', 'tipodepropiedad_Quinta Vacacional',
 'tipodepropiedad_Rancho', 'tipodepropiedad_Terreno', 'tipodepropiedad_Terreno comercial',
 'tipodepropiedad_Villa', 'tipodepropiedad_otro', 'provincia_Aguascalientes',
 'provincia_Baja California Norte', 'provincia_Baja California Sur','provincia_Campeche', 'provincia_Chiapas',
 'provincia_Chihuahua', 'provincia_Coahuila', 'provincia_Colima', 'provincia_Distrito Federal',
 'provincia_Durango','provincia_Edo. de México', 'provincia_Guanajuato', 'provincia_Guerrero',
 'provincia_Hidalgo', 'provincia_Jalisco', 'provincia_Michoacán', 'provincia_Morelos', 'provincia_Nayarit',
 'provincia_Nuevo León', 'provincia_Oaxaca', 'provincia_Puebla','provincia_Querétaro', 'provincia_Quintana Roo',
 'provincia_San luis Potosí', 'provincia_Sinaloa', 'provincia_Sonora', 'provincia_Tabasco', 'provincia_Tamaulipas',
 'provincia_Tlaxcala','provincia_Veracruz', 'provincia_Yucatán', 'provincia_otro', 'zona_Centro', 'zona_Norte ',
 'zona_Sur', 'idf_titulo', 'idf_descripcion', 'peso_titulo', 'peso_descripcion', 'distancia_ciudad_centrica']

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

lgb_m = LightGBMWrapper(**params)

selector = RFE(lgb_m, n_features_to_select=25, verbose=1)
selector.fit(x_train, y_train)

x_train.columns[selector.get_support()]

In [ ]:
display(selector.ranking_[18])
display(selector.ranking_[19])
display(selector.ranking_[37])
display(selector.ranking_[38])
display(list(x_train.columns)[37])

## Recursive Feature Elimination w/ Cross Validation (RFECV)

In [ ]:
from sklearn.feature_selection import RFECV

features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales','lat', 'lng', 'gimnasio',
 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio', 'metrostotales_log',
 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros',
 'metroscubiertos_bins_unif', 'metroscubiertos_bins_perc', 'escomercial', 'promedio_metros_tipo_propiedad',
 'promedio_metros_cub_tipo_propiedad', 'tipo_propiedad_compartida', 'prop_frecuente', 'top_provincia',
 'es_ciudad_centrica', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia',
 'anio', 'mes', 'dia', 'trimestre','dias_desde_datos', 'meses_desde_datos','delincuencia',
 'turismo', 'es_antigua', 'antiguedad_bins_unif', 'antiguedad_bins_perc', 'cantidad_inquilinos', 'tam_ambientes',
 'metros_totales_normalizados', 'metros_cubiertos_normalizados', 'promedio_precio_provincia', 'promedio_provincia_log',
 'promedio_precio_ciudad', 'promedio_ciudad_log', 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad',
 'count_ciudad', 'promedio_id_zona', 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona',
 'count_id_zona', 'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad', 'promedio_precio_tipo_propiedad_ciudad_gen',
 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones',
 'promedio_precio_habitaciones_banos_garages', 'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia',
 'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje', 'tipodepropiedad_Apartamento',
 'tipodepropiedad_Bodega comercial', 'tipodepropiedad_Casa', 'tipodepropiedad_Casa en condominio', 'tipodepropiedad_Casa uso de suelo',
 'tipodepropiedad_Departamento Compartido', 'tipodepropiedad_Duplex', 'tipodepropiedad_Edificio',
 'tipodepropiedad_Inmuebles productivos urbanos', 'tipodepropiedad_Local Comercial', 'tipodepropiedad_Local en centro comercial',
 'tipodepropiedad_Oficina comercial', 'tipodepropiedad_Otros', 'tipodepropiedad_Quinta Vacacional',
 'tipodepropiedad_Rancho', 'tipodepropiedad_Terreno', 'tipodepropiedad_Terreno comercial',
 'tipodepropiedad_Villa', 'tipodepropiedad_otro', 'provincia_Aguascalientes',
 'provincia_Baja California Norte', 'provincia_Baja California Sur','provincia_Campeche', 'provincia_Chiapas',
 'provincia_Chihuahua', 'provincia_Coahuila', 'provincia_Colima', 'provincia_Distrito Federal',
 'provincia_Durango','provincia_Edo. de México', 'provincia_Guanajuato', 'provincia_Guerrero',
 'provincia_Hidalgo', 'provincia_Jalisco', 'provincia_Michoacán', 'provincia_Morelos', 'provincia_Nayarit',
 'provincia_Nuevo León', 'provincia_Oaxaca', 'provincia_Puebla','provincia_Querétaro', 'provincia_Quintana Roo',
 'provincia_San luis Potosí', 'provincia_Sinaloa', 'provincia_Sonora', 'provincia_Tabasco', 'provincia_Tamaulipas',
 'provincia_Tlaxcala','provincia_Veracruz', 'provincia_Yucatán', 'provincia_otro', 'zona_Centro', 'zona_Norte ',
 'zona_Sur', 'idf_titulo', 'idf_descripcion', 'peso_titulo', 'peso_descripcion', 'distancia_ciudad_centrica']

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.001)

lgb_m = LightGBMWrapper(**params)

selector = RFECV(lgb_m, verbose=1)
selector.fit(x_train, y_train)

x_train.columns[selector.get_support()]

In [ ]:
def full_display(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
        display(df)
        
df_ranks = pd.DataFrame(np.array([selector.ranking_, list(x_train.columns)]).T, columns=['rank', 'col'])
df_ranks['rank'] = df_ranks['rank'].astype(int)
full_display(df_ranks.sort_values('rank'))

df_ranks[df_ranks['rank'] == 1]['col'].tolist()

## PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(0.90)

features = ['metrostotales_log', 'metroscubiertos_log', 'porcentaje_metros', 'diferencia_metros',
 'metroscubiertos_bins_perc', 'escomercial', 'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
 'top_provincia', 'promedio_metros_totales_provincia', 'promedio_metros_cubiertos_provincia',
 'mes', 'dia', 'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_unif', 'antiguedad_bins_perc',
 'cantidad_inquilinos', 'tam_ambientes', 'metros_totales_normalizados', 'metros_cubiertos_normalizados',
 'promedio_precio_provincia', 'promedio_precio_ciudad', 'promedio_precio_ciudad_gen', 'varianza_precio_ciudad',
 'count_ciudad', 'promedio_id_zona', 'promedio_id_zona_log', 'promedio_id_zona_gen', 'varianza_id_zona',
 'count_id_zona', 'promedio_precio_tipo_propiedad', 'promedio_precio_tipo_propiedad_ciudad',
 'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad',
 'promedio_por_mes', 'varianza_por_mes', 'promedio_precio_habitaciones', 'promedio_precio_habitaciones_banos_garages',
 'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
 'promedio_precio_hbg_tipo_propiedad_provincia', 'lat_norm', 'lng_norm', 'promedio_precio_booleanos',
 'puntaje', 'idf_titulo', 'idf_descripcion', 'peso_descripcion']

df_train_red = pca.fit_transform(utils.filtrar_features(df_train_f, features))

df_train_red.shape

In [ ]:
len(df_train_red.T)

In [ ]:
np.log(df_train_f['precio']).hist()

In [ ]:
df_train_f['precio'].hist()